In [1]:
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import cPickle as pickle
import pandas as pd
import numpy as np
import scipy.linalg.blas
import datetime
import scipy
import time
from dateutil import rrule 

print "LOADED"

LOADED


In [2]:
def NYSE_holidays2(a, b): 
    # Generate ruleset for holiday observances on the NYSE 
    rs = rrule.rruleset()
    
    # Include all potential holiday observances 
    ###############################################
    
    # New Years Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=12,bymonthday=31, 
                         byweekday=rrule.FR))               
    # New Years Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=1,bymonthday=1))
    # New Years Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=1,bymonthday=2, 
                         byweekday=rrule.MO))                    
    # MLK Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=1,
                         byweekday=rrule.MO(3)))                            
    # Washington's Bday
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=2,
                         byweekday=rrule.MO(3)))                          
    # Good Friday
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,byeaster=-2)) 
    # Memorial Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=5, 
                         byweekday=rrule.MO(-1)))                         
    # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=7,bymonthday=3, 
                         byweekday=rrule.FR))              
    # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=7,bymonthday=4))
    # Independence Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=7,bymonthday=5, 
                         byweekday=rrule.MO))               
    # Labor Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=9, 
                         byweekday=rrule.MO(1)))                          
    # Thanksgiving Day
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=11, 
                         byweekday=rrule.TH(4)))                          
    # Christmas
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=12,bymonthday=24, 
                         byweekday=rrule.FR))                
    # Christmas
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=12,bymonthday=25))
    # Christmas
    rs.rrule(rrule.rrule(rrule.YEARLY,dtstart=a,until=b,bymonth=12,bymonthday=26, 
                         byweekday=rrule.MO))                
    ######################################################
    
    # Exclude potential holidays that fall on weekends 
    rs.exrule(rrule.rrule(rrule.WEEKLY,dtstart=a,until=b,
                          byweekday=(rrule.SA,rrule.SU))) 
    return rs 

def NYSE_tradingdays2(a, b):
    # Generate ruleset for NYSE trading days
    rs = rrule.rruleset() 
    rs.rrule(rrule.rrule(rrule.DAILY,dtstart=a,until=b)) 
    
    # Exclude weekends and holidays 
    rs.exrule(rrule.rrule(rrule.WEEKLY,dtstart=a,byweekday=(rrule.SA,rrule.SU)))
    rs.exrule(NYSE_holidays2(a, b)) 
    
    return rs

print "LOADED"

LOADED


In [3]:
def create_neural_network(neural_network, delay_input=[0], delay_internal=[], delay_output=[]):
    """ Create Neural Network
    Example: 
      network         = [2,3,4,1] network w/ 2 inputs, 2 hidden layers w/ 3 and 4 neurons, 
                                  and 1 linear output layer                             
      delay_input     = [0,1,5] will use inputs of timestep t, t-1 and t-5
      delay_internal  = [1,2,5] adds recurrent connection for output of each layer 
                                in timestep t-1, t-2 and t-5
      delay_output    = [1,3,4] adds recurrent connection for output of each layer in 
                                timestep t-1, t-3 and t-5
    
    Args:
        network:        structure of the neural network [I HL1 HL2 ... HLN OL]
                        number of layers is the length of the list-1
                        number neurons in each layer is the given number
    
        delay_input:    Time delays for NN inputs. 
                        To use only the input of timestep t dIn = [0]
            
        delay_internal: Time delays for recurrent internal connections of NN.
                        dIntern has to be greater than zero (layer output at timestep t-x)!
                        if non-empty list given, recurrent connection from every layer 
                        to itself and every layer before is added
            
        delay_output:   Time delays for recurrent connections of output to first hidden 
                        layer. dOut has to be greater than zero (output at timestep t-x)!
                        if non-empty list given, recurrent connection from NN output to first 
                        hidden layer is added 
    
    Returns:
        
        net: untrained neural network 
    """
    
    network                = {}          
    
    network['delay']       = {'input'    :delay_input, 
                              'internal' :delay_internal, 
                              'output'   :delay_output}   
    
    #Structure
    network['network']     = neural_network  
    
    network['num_layers']  = len(neural_network) - 1  
    
    #structure without inputs
    network['layers']      = neural_network[1:] 
    
    #maximum time delay
    network['max_delay']   = max( max(delay_input, delay_internal, delay_output) )
    
    #initialize random weight vector and specify sets
    network                = create_weight_vector(network)    
    
    #weight vector used for calculation
    network['weight_vect'] = network['w0'].copy()
    
    #number of weights
    network['num_weights'] = len(network['w0'])           
    
    return network

print "LOADED"

LOADED


In [43]:
def create_weight_vector(network):
    """
    Creates random weight vector of NN and defines sets needed for derivative calculation
    
    Returns: 
        neural network
    """
    
    num_layers_network  = network['num_layers']   
    layers              = network['layers'] 
    delay               = network['delay'] 
    inputs              = network['network'][0]
    
    # Input layers or layers w/ internal delay > 0
    input_layers     = [] 
    # Output of layer used for cost func calc or added to input layer w/ delay > 1
    output_layers    = []  
    
    # Connection weight matrix layer m -> layer l w/ delay d
    connection_weight_matrices  = {}  
    bias                        = {}
    input_weight_matrices       = {}
    layers_bkwd_connect_layerM  = {} 
    layers_fwd_connect_layerM   = {} 
    delay_layerM_toL            = {}   
    input_layersU_connect_to    = {}            
    output_layersX_connect_to   = {}
            
    '''Inputs'''
    inputs_connect_layer1      = {} 
    inputs_connect_layer1[1]   = [1]           
    delay_input_layer1         = {}
    delay_input_layer1[1, 1]   = delay['input']
  
    for x in delay_input_layer1[1, 1]:
        # Input-weight matrix set to random values [-0.5,0.5]
        input_weight_matrices[1, 1, x] = np.random.rand(layers[0], inputs) - 0.5  
    
    # First layer is input layer
    input_layers.append(1)  
    
    '''Internal Connection Weight Matrices'''
    for m in range(1, num_layers_network + 1):
        layers_bkwd_connect_layerM[m] = []     
        layers_fwd_connect_layerM[m]  = []
            
        # Forward connects
        if m > 1:
            l = m - 1
            # No delay for forward connects
            delay_layerM_toL[m, l]              = [0]                                            
            connection_weight_matrices[m, l, 0] = np.random.rand(layers[m - 1], 
                                                                 layers[l - 1]) - 0.5 
            
            layers_bkwd_connect_layerM[l].append(m)  
            layers_fwd_connect_layerM[m].append(l)      
    
        # Recursive connects
        for l in range(m, num_layers_network + 1):
            if (m == 1) and (l == num_layers_network):            
                # Delays from output to first layer
                delay_layerM_toL[m, l] = delay['output']          
            else:
                # Internal delays
                delay_layerM_toL[m, l] = delay['internal']       
                
            # All delays for connect l->m    
            for d in delay_layerM_toL[m, l]:                                                     
                connection_weight_matrices[m, l, d] = np.random.rand(layers[m - 1], 
                                                                     layers[l - 1]) - 0.5
                
                # Add if haven't yet
                if (l not in layers_fwd_connect_layerM[m]): 
                    layers_fwd_connect_layerM[m].append(l)  
                    
                # If recurrent connect
                if (l >= m) and(d > 0): 
                    if (m not in input_layers):  
                        input_layers.append(m) 
                        
                    if (l not in output_layers): 
                        output_layers.append(l)
        
        # Create bias vect for layer m
        bias[m] = np.random.rand(layers[m - 1]) - 0.5  
    
    if num_layers_network not in output_layers:
        output_layers.append(num_layers_network)
        
    for u in output_layers:
        input_layersU_connect_to[u] = []
        
        for x in input_layers: 
            # If input layer in lfwd[x] 
            #  and connect x -> u has delay > 0 
            #  and x not yet in inputlayersUconnto[u]
            if  (u in layers_fwd_connect_layerM[x]) and \
                (np.any(np.array(delay_layerM_toL[x, u]) > 0)) and \
                (x not in input_layersU_connect_to[u]):
                
                input_layersU_connect_to[u].append(x)
                
    for x in range(1, num_layers_network + 1):
        output_layersX_connect_to[x] = []
        
        for u in output_layers:
            try:
                # If connect u -> x has delay > 0
                if np.any(np.array(delay_layerM_toL[x, u]) > 0): 
                    output_layersX_connect_to[x].append(u)
            
            except KeyError:
                pass
            
    #Add to network
    network['output_layers']               = output_layers
    network['input_layers']                = input_layers
    network['delay_layerM_toL']            = delay_layerM_toL
    network['delay_input_layer1']          = delay_input_layer1
    network['layers_bkwd_connect_layerM']  = layers_bkwd_connect_layerM
    network['layers_fwd_connect_layerM']   = layers_fwd_connect_layerM
    network['inputs_connect_layer1']       = inputs_connect_layer1
    network['input_layersU_connect_to']    = input_layersU_connect_to
    network['output_layersX_connect_to']   = output_layersX_connect_to
    
    network['w0'] = convert_matrices_to_vector(network, 
                                               input_weight_matrices, 
                                               connection_weight_matrices, 
                                               bias)
    return network

print "LOADED"

LOADED


In [44]:
def convert_matrices_to_vector(network, input_weight_matrices, connection_weight_matrices, bias):
    """
    Converts input weight matrices(IW), connection weight matrices(LW) and bias 
    vectors(b) to weight vector(w)
    """
    
    delay_layerM_toL           = network['delay_layerM_toL']    
    delay_input_layer1         = network['delay_input_layer1']    
    inputs_connect_layer1      = network['inputs_connect_layer1']     
    layers_fwd_connect_layerM  = network['layers_fwd_connect_layerM']   
    num_layers_network         = network['num_layers']     
    weight_vect                = np.array([])
    
    # Input weights
    for m in range(1, num_layers_network + 1): 
        if m == 1:
            for i in inputs_connect_layer1[m]:
                for d in delay_input_layer1[m, i]:
                    weight_vect = np.append(weight_vect, 
                                            input_weight_matrices[m, i, d].flatten('F'))
                    
        # Internal connect weights
        for l in layers_fwd_connect_layerM[m]:
            for d in delay_layerM_toL[m, l]:
                weight_vect = np.append(weight_vect, 
                                        connection_weight_matrices[m, l, d].flatten('F'))
                
        # Bias weights
        weight_vect = np.append(weight_vect, bias[m])
    
    return weight_vect

print "LOADED"

LOADED


In [45]:
def convert_vector_to_matrices(network):
    """
    Converts weight vector w to Input Weight matrices IW, connection weight 
    matrices LW and bias vectors b
    """
    
    delay_layerM_toL            = network['delay_layerM_toL']       
    delay_input_layer1          = network['delay_input_layer1']       
    inputs_connect_layer1       = network['inputs_connect_layer1']        
    layers_fwd_connect_layerM   = network['layers_fwd_connect_layerM']      
    num_layers_network          = network['num_layers']        
    layers                      = network['layers']   
    inputs                      = network['network'][0]    
    weight_vect_temp            = network['weight_vect'].copy() 
    input_weight_matrices       = {}              
    connection_weight_matrices  = {}              
    bias                        = {}              
    
    for m in range(1, num_layers_network + 1):
        # Input weights
        if m == 1:
            for i in inputs_connect_layer1[m]:
                for d in delay_input_layer1[m, i]:
                    weight_i                       = inputs * layers[m - 1]
                    vec                            = weight_vect_temp[0 : weight_i]
                    weight_vect_temp               = weight_vect_temp[weight_i :]
                    
                    input_weight_matrices[m, i, d] = np.reshape(vec, (layers[m - 1], 
                                                                      len(vec) / layers[m - 1]), 
                                                                      order = 'F')
        
        # Internal connect weights
        for l in layers_fwd_connect_layerM[m]:
            for d in delay_layerM_toL[m, l]:
                weight_i                            = layers[l - 1] * layers[m - 1]
                vec                                 = weight_vect_temp[0 : weight_i]
                weight_vect_temp                    = weight_vect_temp[weight_i :]
                
                connection_weight_matrices[m, l, d] = np.reshape(vec, (layers[m - 1], 
                                                                       len(vec) / layers[m - 1]), 
                                                                       order = 'F')
        
        # Bias vector of layer m
        weight_i         = layers[m - 1]
        bias[m]          = weight_vect_temp[0 : weight_i]
        weight_vect_temp = weight_vect_temp[weight_i :]

    return input_weight_matrices, connection_weight_matrices, bias

print "LOADED"

LOADED


In [46]:
def get_network_output(network_inputs, 
                       network, 
                       input_weight_matrices, 
                       connection_weight_matrices, 
                       bias, 
                       layer_outputs = {}, 
                       num_prev_data_pts = 0):
    """
    Calculates network output for given inputs
    """
    
    delay_layerM_toL              = network['delay_layerM_toL']                                 
    delay_input_layer1            = network['delay_input_layer1']                                 
    inputs_connect_layer1         = network['inputs_connect_layer1']                                  
    layers_fwd_connect_layerM     = network['layers_fwd_connect_layerM']                                
    num_layers_network            = network['num_layers']                                  
    outputs                       = network['network'][-1]                             
    sum_output_layers             = {}                                        
    num_of_input_datapts          = network_inputs.shape[1]                        
    network_output                = np.zeros((outputs, num_of_input_datapts)) 
    
    # For all datapoints
    for q in range(num_prev_data_pts + 1, num_of_input_datapts + 1): 
        layer_outputs[q, 1] = 0
        # For all layers
        for m in range(1, num_layers_network + 1): 
            # Sum output datapoint q, layer m
            sum_output_layers[q, m] = 0         
            
            # Input weights
            if m == 1:
                for i in inputs_connect_layer1[m]:
                    for d in delay_input_layer1[m, i]:
                        if (q - d) > 0:
                            sum_output_layers[q, m] += np.dot(input_weight_matrices[m, i, d], 
                                                              network_inputs[:, q - d - 1])
                            
            # Connect weights
            for l in layers_fwd_connect_layerM[m]:
                for d in delay_layerM_toL[m, l]:
                    if (q - d) > 0:
                        sum_output_layers[q, m] += np.dot(connection_weight_matrices[m, l, d], 
                                                          layer_outputs[q - d, l])
            # Bias
            sum_output_layers[q, m] += bias[m]
            
            # Calc layer output
            if m == num_layers_network:
                # Linear layer for output
                layer_outputs[q, num_layers_network] = sum_output_layers[q, num_layers_network] 
            else:
                layer_outputs[q, m] = np.tanh(sum_output_layers[q, m])
        
        network_output[:, q - 1] = layer_outputs[q, num_layers_network]
    
    network_output = network_output[:, num_prev_data_pts :]
    
    return network_output, sum_output_layers, layer_outputs

print "LOADED"

LOADED


In [47]:
def RTRL(network, data):
    """ 
    Jacobian Matrix    == derivatives of error with respect to weight vector
    Mean Squared Error == MSE of network compared to training data
    Error Vector       == difference of network output and target data
    """
    
    #data == training data
    network_inputs               = data['inputs']      
    network_outputs              = data['outputs']      
    layer_outputs                = data['layer_outputs']
    num_prev_data_pts            = data['q0'] 
    
    delay_layerM_toL             = network['delay_layerM_toL']      
    delay_input_layer1           = network['delay_input_layer1']
    
    inputs_connect_layer1        = network['inputs_connect_layer1']       
    layers_fwd_connect_layerM    = network['layers_fwd_connect_layerM']     
    layers_bkwd_connect_layerM   = network['layers_bkwd_connect_layerM']     
    num_layers_network           = network['num_layers']   
    
    inputs                       = network['network'][0]   
    outputs                      = network['network'][-1]
    
    layers                       = network['layers']  
    input_layers                 = network['output_layers']       
    output_layers                = network['input_layers']       
    output_layersX_connect_to    = network['output_layersX_connect_to']   
    
    input_weight_matrices, connection_weight_matrices, bias = convert_vector_to_matrices(network)
    
    # 1. Calc network output
    network_out, sum_output_layers, layer_outputs = get_network_output(
        network_inputs, 
        network, 
        input_weight_matrices,                                     
        connection_weight_matrices, 
        bias, 
        layer_outputs     = layer_outputs, 
        num_prev_data_pts = num_prev_data_pts)
    
    # 2. Calc cost func
    error_matrix       = network_outputs - network_out                                             
    error_vector       = np.reshape(error_matrix, (1, np.size(error_matrix)), order = 'F')[0]
    #mean_squared_error = np.dot(error_vector, error_vector.transpose())
    #mean_absolute_error = np.sum(np.absolute(error_vector))
    mean_absolute_error = np.average(np.absolute(error_vector))
    
    # 3. Backpropagation RTRL
    num_of_input_datapts            = network_inputs.shape[1]                       
    num_of_datapts_without_old_data = num_of_input_datapts - num_prev_data_pts 
    
    
    deriv_layer_outputsOfU_respect_bias_vect                     = {} 
    deriv_layer_outputsOfU_respect_input_weight_matrices         = {} 
    deriv_layer_outputsOfU_respect_connectection_weight_matrices = {}
    deriv_layer_outputs_respect_weight_vect                      = {} 
    sensitivity_matrix                                           = {} 
    layersM_with_existing_sensitivity_matrix                     = {} 
    input_layersX_with_existing_sensitivity_matrix               = {} 
                                                                        
    # Init
    jacobian_matrix = np.zeros((num_of_datapts_without_old_data * layers[-1], 
                                network['num_weights']))
    
    for q in range(1, num_prev_data_pts + 1):
        for u in input_layers:
            deriv_layer_outputsOfU_respect_connectection_weight_matrices[q, u] = np.zeros(
                (layers[u - 1], 
                network['num_weights']))

    # Begin RTRL
    for q in range(num_prev_data_pts + 1, num_of_input_datapts + 1):
        # Init, set needed for calculating sensitivities
        input_layers_ = [] 
        for u in input_layers:
            layersM_with_existing_sensitivity_matrix[u]       = []
            input_layersX_with_existing_sensitivity_matrix[u] = []
            deriv_layer_outputs_respect_weight_vect[q, u]     = 0
        
        # Calc sensitivity matrices, decrement m in backprop order
        for m in range(num_layers_network, 0, -1): 
            for u in input_layers_:
                # Sensitivity Matrix layer u -> m
                sensitivity_matrix[q, u, m] = 0 
                
                for l in layers_bkwd_connect_layerM[m]:
                    #recursive calculation of Sensitivity Matrix layer u -> m
                    sensitivity_matrix[q, u, m] += np.dot(
                        np.dot(sensitivity_matrix[q, u, l], connection_weight_matrices[l, m, 0]),
                        np.diag(1 - (np.tanh(sum_output_layers[q, m])) ** 2)) 
                      
                if m not in layersM_with_existing_sensitivity_matrix[u]:
                    layersM_with_existing_sensitivity_matrix[u].append(m) 
                    
                    if m in output_layers:
                        input_layersX_with_existing_sensitivity_matrix[u].append(m)
                        
            if m in input_layers:
                # Output layer is linear, no transfer function
                if m == num_layers_network: 
                    sensitivity_matrix[q, m, m] = np.diag(np.ones(outputs)) 
                else:
                    sensitivity_matrix[q, m, m] = np.diag(1 - (np.tanh(sum_output_layers[q, m])) 
                                                                              ** 2)
                
                # Add m to U'
                input_layers_.append(m) 
                layersM_with_existing_sensitivity_matrix[m].append(m)
                
                if m in output_layers:
                    input_layersX_with_existing_sensitivity_matrix[m].append(m)
          
        '''Calc derivs, static deriv calc'''
        for u in sorted(input_layers): 
            # Static deriv vector: explicit deriv layer outputs w/ respect to weight vect
            deriv_layer_outputs_respect_weight_vect_ = np.empty((layers[u - 1], 0))
            
            # Input weights
            for m in range(1, num_layers_network + 1): 
                if m == 1:
                    for i in inputs_connect_layer1[m]:
                        for d in delay_input_layer1[m, i]:
                            # If no sensivity matrix exists or d >= q: deriv is zero
                            if ((q, u, m) not in sensitivity_matrix.keys()) or (d >= q): 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m, i, d] = \
                                    np.kron(network_inputs[:, q - d - 1].transpose(), 
                                    np.zeros((layers[u - 1], layers[m - 1])))
                            else: 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m,i,d] = \
                                    np.kron(network_inputs[:, q - d - 1].transpose(), 
                                    sensitivity_matrix[q, u, m])

                            # Append to static deriv vect
                            deriv_layer_outputs_respect_weight_vect_ = np.append(
                                deriv_layer_outputs_respect_weight_vect_, 
                                deriv_layer_outputsOfU_respect_input_weight_matrices[m, i, d], 
                                1) 
        
                # Connect weights
                for l in layers_fwd_connect_layerM[m]:
                    for d in delay_layerM_toL[m, l]:
                        # If no sensivity matrix exists or d >= q: deriv is zero
                        if ((q, u, m) not in sensitivity_matrix.keys()) or (d >= q): 
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d]=\
                                np.kron(layer_outputs[q, l].transpose(), np.zeros((
                                            layers[u - 1], layers[m - 1])))
                        else:
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d]=\
                                np.kron(layer_outputs[q - d, l].transpose(), 
                                        sensitivity_matrix[q, u, m]) 

                        # Append to static deriv vect
                        deriv_layer_outputs_respect_weight_vect_ = np.append(
                            deriv_layer_outputs_respect_weight_vect_, 
                            deriv_layer_outputsOfU_respect_connectection_weight_matrices[m,l,d], 
                            1) 
                        
                # Bias weights
                if ((q, u, m) not in sensitivity_matrix.keys()):
                    # Deriv is zero
                    deriv_layer_outputsOfU_respect_bias_vect[m] = np.zeros((layers[u - 1], 
                                                                            layers[m - 1])) 
                else:
                    deriv_layer_outputsOfU_respect_bias_vect[m] = sensitivity_matrix[q, u, m] 

                # Append to static deriv vect
                deriv_layer_outputs_respect_weight_vect_ = np.append(
                                        deriv_layer_outputs_respect_weight_vect_, 
                                        deriv_layer_outputsOfU_respect_bias_vect[m], 
                                        1) 
            
            '''Dynamic deriv calc'''
            dyn_deriv_sum_allX = 0
            for x in input_layersX_with_existing_sensitivity_matrix[u]:
                # Sum of all u_
                sum_u_ = 0 
                
                for u_ in output_layersX_connect_to[x]:
                    # Sum of all d
                    sum_d = 0 
                    
                    for d in delay_layerM_toL[x, u_]:
                        # Delays > 0 and < q
                        if (q - d > 0) and (d > 0): 
                            sum_d += np.dot(connection_weight_matrices[x, u_, d], 
                                            deriv_layer_outputs_respect_weight_vect[q - d, u_])

                    sum_u_ += sum_d

                if sum_u_ is not 0:
                    # Sum up dynamic deriv
                    dyn_deriv_sum_allX += np.dot(sensitivity_matrix[q, u, x], sum_u_) 
            
            # Static + dynamic deriv, total deriv output layer u with respect to w
            deriv_layer_outputs_respect_weight_vect[q, u] = \
                    deriv_layer_outputs_respect_weight_vect_ + dyn_deriv_sum_allX 
        # Jacobian matrix
        jacobian_matrix[range(((q - num_prev_data_pts) - 1) * outputs, 
                              (q - num_prev_data_pts) * outputs), :] = \
            -deriv_layer_outputs_respect_weight_vect[q, num_layers_network]
            
        ############!!!!#########!!!#################!@!@!@!@!#####
        if q >= 7:
            your_keys = [(q,2),(q,1),(q-1,2),(q-1,1),(q-2,2),(q-2,1),(q-3,2),
                         (q-3,1),(q-4,2),(q-4,1),(q-5,2),(q-5,1),(q-6,2),(q-6,1)]
            deriv_layer_outputs_respect_weight_vect = {
                key: deriv_layer_outputs_respect_weight_vect[key] 
                for key in deriv_layer_outputs_respect_weight_vect.keys() 
                if key in your_keys}
            sens_keys = [(q,  2,2),(q,  2,1),(q,  1,1),(q-1,2,2),(q-1,2,1),(q-1,1,1),
                         (q-2,2,2),(q-2,2,1),(q-2,1,1),(q-3,2,2),(q-3,2,1),(q-3,1,1),
                         (q-4,2,2),(q-4,2,1),(q-4,1,1),(q-5,2,2),(q-5,2,1),(q-5,1,1),
                         (q-6,2,2),(q-6,2,1),(q-6,1,1)]
            sensitivity_matrix ={key2: sensitivity_matrix[key2] 
                             for key2 in sensitivity_matrix.keys() 
                             if key2 in sens_keys}
        ############!!!!#########!!!#################!@!@!@!@!#####
        
    #return jacobian_matrix, mean_squared_error, error_vector
    return jacobian_matrix, mean_absolute_error, error_vector

print "LOADED"

LOADED


In [48]:
def prepare_data(network_inputs, 
                 network_targets, 
                 network, 
                 prev_input_data0  = None, 
                 prev_output_data0 = None):
    """
    Prepare input data for network training and check for errors
    
    Returns: 
        dict containing data for training or calculating output
    """
    
    # Convert inputs and outputs to 2D array, if 1D array is given
    if network_inputs.ndim == 1:
        network_inputs = np.array([network_inputs])
    
    if network_targets.ndim == 1:
        network_targets = np.array([network_targets]) 
        
    # Check if input and output data match structure of network
    if np.shape(network_inputs)[0] != network['network'][0]:
        raise ValueError("Dimension of input data doesn't match # of inputs of network")
    
    if np.shape(network_targets)[0] != network['network'][-1]:
        raise ValueError("Dimension of output data doesn't match # of outputs of network")
    
    if np.shape(network_inputs)[1] != np.shape(network_targets)[1]:
        raise ValueError("Input and output data must have same # of datapoints")
        
    # Check if prev data given, convert input and output to 2D array, if 1D array given
    if (prev_input_data0 is not None) and (prev_output_data0 is not None): 
        if prev_input_data0.ndim == 1:
            prev_input_data0 = np.array([prev_input_data0])
        
        if prev_output_data0.ndim == 1:
            prev_output_data0 = np.array([prev_output_data0])
            
        # Check if input and output data match structure of network
        if np.shape(prev_input_data0)[0] != network['network'][0]:
            raise ValueError("Dimension of prev input data(p0) doesn't match # inputs of network")
        
        if np.shape(prev_output_data0)[0] != network['network'][-1]:
            raise ValueError("Dimension of prev output data(y0) doesn't match # outputs network")
        
        if np.shape(prev_input_data0)[1] != np.shape(prev_output_data0)[1]:
            raise ValueError("Prev input and output data must have same # of datapoints(q0)")
            
        num_prev_data_pts = np.shape(prev_input_data0)[1] 
        
        # Init layer outputs
        layer_outputs = {} 
        
        for i in range(1, num_prev_data_pts + 1):
            for j in range(1, network['num_layers']):
                # Layer ouputs of hidden layers are unknown -> set to zero
                layer_outputs[i, j] = np.zeros(network['network'][-1]) 
            
            # Set layer ouputs of output layer
            layer_outputs[i, network['num_layers']] = prev_output_data0[:, i - 1] / \
                network['normY'] 
            
        # Add prev inputs and outputs to input/output matrices
        updated_inputs  = np.concatenate([prev_input_data0, network_inputs], axis=1)
        updated_outputs = np.concatenate([prev_output_data0, network_targets], axis=1)
    
    # Keep inputs and outputs as is and set q0 and a to default vals
    else: 
        updated_inputs    = network_inputs.copy()
        updated_outputs   = network_targets.copy()
        num_prev_data_pts = 0
        layer_outputs     = {}
        
    # Normalize
    inputs_normed  = updated_inputs.copy()
    outputs_normed = updated_outputs.copy()
    
    if 'normP' not in network.keys():
        normInp = np.ones(np.shape(updated_inputs)[0])

        for p in range(np.shape(updated_inputs)[0]):
            normInp[p]       = np.max([np.max(np.abs(updated_inputs[p])), 1.0])
            inputs_normed[p] = updated_inputs[p] / normInp[p]

        normOut = np.ones(np.shape(updated_outputs)[0])

        for y in range(np.shape(updated_outputs)[0]):
            normOut[y]        = np.max([np.max(np.abs(updated_outputs[y])), 1.0])
            outputs_normed[y] = updated_outputs[y] / normOut[y] 
            
        network['normP'] = normInp
        network['normY'] = normOut
   
    else:
        for p in range(np.shape(updated_inputs)[0]):
            inputs_normed[p] = updated_inputs[p] / network['normP'][p]
            
        normOut = np.ones(np.shape(network_targets)[0])
        
        for y in range(np.shape(updated_outputs)[0]):
            outputs_normed[y] = updated_outputs[y] / network['normY'][y]
            
    # Create data dict
    data                   = {}
    data['inputs']         = inputs_normed
    data['outputs']        = outputs_normed
    data['layer_outputs']  = layer_outputs
    data['q0']             = num_prev_data_pts    
    
    return data, network

print "LOADED"

LOADED


In [50]:
def train_LM(nn_inputs, 
             nn_outputs, 
             net, 
             iteration_max = 10, 
             MSE_stop      = 1e-6, 
             damp_factor   = 3.0, 
             damp_const    = 10.0, 
             verbose       = False):
    """
    Levenberg-Marquardt(LM) algorithm
      - Least-squares estimation of nonlinear parameters
      
    Args:
        iteration_max:  max # of iterations
        MSE_stop:       termination error, training stops when MSE <= MSE_stop
        damp_const:     constant to adapt damping factor of LM
        damp_factor:    damping factor of LM
    
    Returns:
        net:            trained Neural Network 
    """
    
    data, net = prepare_data(nn_inputs, nn_outputs, net)
    
    # Calc for first iteration
    Jacobian, Mean_absolute_error, error_vect = RTRL(net, data)
    
    # Vect for error history
    iteration               = 0
    ErrorHistory            = np.zeros(iteration_max + 1) 
    ErrorHistory[iteration] = Mean_absolute_error
    
    if verbose:
        print('Iteration: ',    iteration, 
              'Error: ',        Mean_absolute_error, 
              'scale factor: ', damp_factor)
    
    seconds, minutes, hours = 60, 60, 6
    t_end = time.time() + (seconds * minutes * hours)

    # Run loop until either max iterations or MSE_stop reached
    while True:
        #####
        JTJ = scipy.linalg.blas.dgemm(alpha=1.0, a=Jacobian.T, b=Jacobian.T, trans_b=True)
        #####
        weight_vect = net['weight_vect']
        
        # Repeat until optimizing step successful
        while True:
            seconds2, minutes2, hours2 = 60, 60, 1.5
            t_end2 = time.time() + (seconds2 * minutes2 * hours2)
            
            gradient = np.dot(Jacobian.transpose(), error_vect)

            # Calc scaled inverse Hessian
            try:
                #####
                scaled_inv_hessian = scipy.linalg.inv(JTJ + damp_factor * 
                                                      np.eye(net['num_weights'])) 
                #####
            except np.linalg.LinAlgError:
                # Not invertible, do small step in gradient direction
                weight_delta = 1.0 / 1e10 * gradient
            else:
                # Calc weight modification
                weight_delta = np.dot(-scaled_inv_hessian, gradient)
            
            # New weight vect
            net['weight_vect'] = weight_vect + weight_delta  
            new_mean_absolute_error = calc_error(net, data)
            
            # If optimization step successful, adapt scale factor, then go next iteration
            if new_mean_absolute_error < Mean_absolute_error:
                damp_factor = damp_factor / damp_const 
                break 
            else:
                damp_factor = damp_factor * damp_const
                
            if (time.time() > t_end) or (time.time() > t_end2):
                break
        if (time.time() > t_end) or (time.time() > t_end2):
            break
        
        # Calc for next iteration
        Jacobian, Mean_absolute_error, error_vect = RTRL(net, data)
        iteration += 1
        ErrorHistory[iteration] = Mean_absolute_error
        
        if verbose:
            print('Iteration: ',    iteration,
                  'Error: ',        Mean_absolute_error,
                  'scale factor: ', damp_factor)

        # Check if termination condition hit
        if iteration >= iteration_max:
            print('Max # of iterations reached')
            break
        elif Mean_absolute_error <= MSE_stop:
            print('Termination error reached')
            break

    net['ErrorHistory'] = ErrorHistory[:iteration]
    return net

print "DONE"

DONE


In [51]:
def calc_error(network, data):
    """
    Returns:
        MAE of network compared to training data
    """
    
    network_inputs    = data['inputs']
    network_outputs   = data['outputs'] 
    layer_outputs     = data['layer_outputs']
    num_prev_data_pts = data['q0'] 
    
    input_weight_matrices, connection_weight_matrices, bias = convert_vector_to_matrices(network)
    
    network_out, sum_output_layers, layer_outputs = get_network_output(
        network_inputs, 
        network, 
        input_weight_matrices,                                     
        connection_weight_matrices, 
        bias, 
        layer_outputs     = layer_outputs, 
        num_prev_data_pts = num_prev_data_pts)

    # Outputs_delta = error matrix
    outputs_delta       = network_outputs - network_out 
    error_vect          = np.reshape(outputs_delta, (1, np.size(outputs_delta)), order='F')[0]
    Mean_absolute_error = np.average(np.absolute(error_vect))

    #return Mean_squared_error
    return Mean_absolute_error

print "LOADED"

LOADED


In [52]:
def NNOut(inputs, net, P0 = None, Y0 = None):
    """
    Calculates network output for given inputs
    
    Args:
        P0: prev input data
        Y0: prev output data
        
    Returns:
    Y_NN: Neural Network output for input P
    """
    
    outputs   = np.zeros((net['layers'][-1], np.size(inputs) / net['network'][0]))
    data, net = prepare_data(inputs, outputs, net)
    input_weight_matrices, connection_weight_matrices, bias = convert_vector_to_matrices(net)
    
    network_out = get_network_output(
        data['inputs'], 
        net, 
        input_weight_matrices,                                     
        connection_weight_matrices, 
        bias, 
        layer_outputs     = data['layer_outputs'], 
        num_prev_data_pts = data['q0'])[0]

    # Scale normalized output
    network_out_scaled = network_out.copy()
    for y in range(np.shape(network_out)[0]):
        network_out_scaled[y] = network_out[y] * net['normY'][y]
    
    if np.shape(network_out_scaled)[0] == 1:
        network_out_scaled = network_out_scaled[0]
    
    return network_out_scaled

print "LOADED"

LOADED


In [53]:
def create_learntest(num, key, out_num, net=False, keyonly=False):
    """
    Retrieve our dataframes with all of our indicators for our 6 helper indexes/funds that 
    help in the prediction process for a certain company we choose to predict on. The 6 
    indexes are the Dow Jones Index, the S&P 500 Index, the Nasdaq Composite, 
    United States Oil Fund, the SPDR S&P 500 ETF, and SPDR Gold Shares. 
    
    We take these dataframes, and concatenate to our stock we will predict for,
    calling it with a stock key that we feed in through a parameter. 
    
    We then adjust for missing dates that many of the 6 helper stocks lack but that many
    stocks do have, removing those certain dates from our data. We then take the list of 
    indicators we are going to use by loading a list with all of the 222 names of each 
    indicator. There are over 2000 possible indicators, we chose only 222 for memory and
    computation costs of using more than that. 
    
    You can change the indicators as you like, just as long as it's a list of indicator 
    names as strings. We get our 222 indicator dataframe then set our output as our stock
    choices return values at 1 minute intervals. 
    
    We then set up our learn/test lists which have to be lists due to better efficiency
    using numpy rather than pandas dataframes.
    
    Return the inputs, outputs, test inputs, test outputs, and the neural net if we already
    have one set up.
    """
    # If false,we're utilizing helper indexes/etfs/natural-resources for prediction process
    if keyonly == False:
        # Grab our pickled files from our NewBase/STOCKSYMBOL/ directories
        # the num is referring to one of fifteen parts to each company indicator dataframe
        opp       = open('NewBase/^GSPC/^GSPC_df'+str(num)+'.pickle', 'rb')
        opp2      = open('NewBase/^IXIC/^IXIC_df'+str(num)+'.pickle', 'rb')
        opp3      = open('NewBase/^DJI/^DJI_df'+str(num)+'.pickle', 'rb')
        opp4      = open('NewBase/GLD/GLD_df'+str(num)+'.pickle', 'rb')
        opp5      = open('NewBase/USO/USO_df'+str(num)+'.pickle', 'rb')
        opp6      = open('NewBase/SPY/SPY_df'+str(num)+'.pickle', 'rb')
        opp7      = open('NewBase/'+key+'/'+key+'_df'+str(num)+'.pickle', 'rb')
        # Also grab the indicator list that contains all the indicators we'll be using from
        # our indicator dataframes since we can't process that many indicators
        #opp8      = open('Pickles/final_lst222'+key+'.pickle','rb')
        opp8      = open('Pickles/final_lst90'+key+'.pickle','rb')
        gspc      = pickle.load(opp)
        ixic      = pickle.load(opp2)
        dji       = pickle.load(opp3)
        gld       = pickle.load(opp4)
        uso       = pickle.load(opp5)
        spy       = pickle.load(opp6)
        df        = pickle.load(opp7)
        final_lst = pickle.load(opp8)
        opp.close()
        opp2.close()
        opp3.close()
        opp4.close()
        opp5.close()
        opp6.close()
        opp7.close()
        opp8.close()

        # Combine the dataframes into a single dataframe, making sure there aren't any
        # duplicate rows which happened curiously a few times during testing.
        comp_lst = [gspc,ixic,dji,gld,uso,spy,df]
        for x in range(7):
            comp_lst[x] = comp_lst[x].T.groupby(level=0).first().T
        new_df = pd.concat(comp_lst, axis=1).fillna(method='bfill').fillna(method='ffill')

        # Remove these dates because some dataframes contain data from them but others
        # don't and could affect calculations with up to 390 consecutive incorrect values
        dates = ['2013-08-28', '2013-10-28', '2014-02-12', '2014-02-18', '2014-10-02', 
                 '2014-10-06', '2014-10-08', '2014-10-09', '2014-10-13', '2014-10-14', 
                 '2014-10-15', '2014-10-20', '2015-01-14', '2015-04-21', '2015-05-18', 
                 '2015-06-08', '2015-07-08', '2015-08-20', '2015-08-31', '2015-09-08', 
                 '2016-02-08', '2016-03-15', '2016-03-21', '2016-03-22', '2016-04-13', 
                 '2016-06-15', '2015-03-30', '2015-05-05', '2014-02-25']
        for each in dates:
            try:
                nums   = new_df.index.get_loc(each)
                new_df = new_df.drop(new_df.index[nums.start:nums.stop])
            except:
                pass
    else:
        # If true, we only use indicators from the stock we're predicting on
        opp       = open('NewBase/'+key+'/'+key+'_df'+str(num)+'.pickle', 'rb')
        opp2      = open('Pickles/final_lst216'+key+'only.pickle','rb')
        new_df    = pickle.load(opp)
        final_lst = pickle.load(opp2)
        opp.close()
        opp2.close()
    
    new_df        = new_df.T.groupby(level=0).first().T
    # Create the output indicator name so we can call it from our indicator df
    rets_name     = key+'_rets'+str(out_num)
    # Get a dataframe with just our indicators for training/testing, and then
    # the output column, learning_rets

    result        = new_df[final_lst]
    learning_rets = new_df[rets_name]
    
    length        = len(result)
    # Adjust the input/output so inputs/output are in sync. If not, we'd be predicting
    # for for the incorrect returns.
    result        = result.iloc[:length-(out_num+1)]
    learning_rets = learning_rets.iloc[(out_num+1):]
    
    # If already have a neural net trained on previous data, net will be true. Then we
    # set our net_var to the already trained net in prep to train further with it.
    net_var = []
    if net == True:
        if keyonly == False:
            opp = open('Pickles/net222attrs'+key+'.pickle','rb')
        else:
            opp = open('Pickles/net216attrs'+key+'only.pickle','rb')
        net_var = pickle.load(opp)
        opp.close()
    
    # Set up our test input/output dataframes
    result_test   = result.copy(deep=True)
    testing_rets  = learning_rets.copy(deep=True)
    
    # Leave the last 205 records for testing net w/ test data
    sml = 0
    mid = len(result) - 205
    
    # Convert dataframes to numpy arrays for speed/efficiency purposes
    learning_outputs = learning_rets.iloc[sml:mid]
    testing_outputs  = testing_rets.iloc[mid:mid+200]
    learning_inputs  = result.iloc[sml:mid]
    testing_inputs   = result_test[mid:mid+200]

    inputs           = learning_inputs.values.T
    outputs          = learning_outputs.values.T
    test_inputs      = testing_inputs.values.T
    test_outputs     = testing_outputs.values.T

    print mid
    return inputs, outputs, test_inputs, test_outputs, key, net_var

print "LOADED"

LOADED


In [56]:
def start_training(inputs, outputs, key, net=[], keyonly=False):
    """
    If we are using an already created neural network and training futher on it,
    we'll set net2 to net, else we create a neural network with 222 inputs, 12
    hidden neurons, and 1 output.
    
    If we are using our second example indicator list that only contains 
    indicators from our prediction stock rather than both the stock and our
    base etfs/indexes indicators, we'll use our 216 inputs list, and dump
    that file to our net216attrsKEYonly.pickle file.
    """
    # If we don't already have a trained network for further training, create new one
    if net == []:
        if keyonly == False:
            net = create_neural_network([222, 12, 1], 
                       delay_input    = [0,1,2,3,4],        
                       delay_internal = [1,2,3,4],         
                       delay_output   = [1,2,3,4])
        else:
            net = create_neural_network([216, 12, 1], 
                       delay_input    = [0,1,2,3,4],        
                       delay_internal = [1,2,3,4],         
                       delay_output   = [1,2,3,4])
    elif net == 0:
        net = create_neural_network([90, 24, 1], 
                       delay_input    = [0,1,2,3],        
                       delay_internal = [1,2,3],         
                       delay_output   = [1,2,3])
    else:
        net = net
    
    # Return our new trained network
    net = train_LM(inputs, outputs, net, verbose=True, iteration_max=6, MSE_stop=1e-6)

    # Dump that network into a pickled file
    #if keyonly == False:
    #    opp = open('Pickles/net222attrs'+key+'.pickle','wb')
    #else:
    #    opp = open('Pickles/net216attrs'+key+'only.pickle','wb')
    #pickle.dump(net, opp)
    #opp.close()
    
    # Return net for testing/graphing
    return net

print "LOADED"

LOADED


In [55]:
def graph_predictions(test_inputs, test_outputs, net, beg, end):
    """
    Graph the predicted stock return value compared to the actual return value.
    beg is the row your testing, end is the ending row. For example, if 
    beg == 0, and end == 50, we'll look at the predictions for the first 50
    rows of the test inputs.
    """
    # Create our predictions
    ytest  = NNOut(test_inputs, net, P0=None, Y0=None)
    
    # On how much data will we graph
    total  = end - beg
    
    # Graph our predicted vs actual output
    fig = plt.figure()
    plt.plot(range(total), ytest[beg:end], 'b-', label='Ytest')
    plt.plot(range(total), test_outputs[beg:end], 'r', label='Ytrue')
    fig.suptitle('Predicted VS Actual Output', fontsize=20)
    plt.xlabel('Timestamp', fontsize=18)
    plt.ylabel('Return Value', fontsize=16)
    blue_patch = mpatches.Patch(color='blue', label='Predicted Output')
    red_patch  = mpatches.Patch(color='red', label='Actual Output')
    plt.legend(handles=[red_patch, blue_patch])
    plt.show()
    return

print "LOADED"

LOADED


In [40]:
inputs,outputs,test_inputs,test_outputs,key,net=create_learntest(2,'GOOG',0, net=True)
#inputs,outputs,test_inputs,test_outputs,key,net=create_learntest(5,'AAPL',0, net=True, keyonly=True)

24818


In [ ]:
net = start_training(inputs, outputs, key, net=0)
#net = start_training(inputs, outputs, key, net=net, keyonly=True)

In [18]:
graph_predictions(test_inputs, test_outputs, net, 55, 105)

In [34]:
lst = ['GOOG','^GSPC','^IXIC','^DJI','GLD','USO','SPY']
lst2 = ['rsi', 'vol', 'sma', 'cci', 'per', 'mom', 'bol', 
           'aro', 'mac', 'mactwo', 'adx', 'kdo', 'rets']
test = []
for key in lst:
    for nm in lst2:
        if (key+'_'+nm+'0') != 'GOOG_rets0':
            test.append(key+'_'+nm+'0')